$$
y = w_1X_1 + w_2X_2 + ... + w_nX_n + b
$$
$$
y = wX + b
$$

$$
y = w_0X_0 + w_1X_1 + w_2X_2 + ...+ w_nX_n
$$
$$
y = w X
$$

$y$ - целевое значение \
$X$ - Пространство признаков (включая фиктивный единичный столбец)\
$w$ - Вес при признаках (Коэффициент наклона) `coef_`\
$b(w_0)$ - Свободный член (Коэффициент сдвига) `intercept_`

# Least Squares

Нахождение коэффициентов линейной регрессии методом **минимизации среднеквадратичной ошибки:**\
$w = \arg\min_{w}E(w, X, y), \hspace{1cm} ||Xw-y||^2 \rightarrow min$

---

## Normal Equition

$$
E(w, X, y) = \frac{1}{2n}\sum_{i=1}^n (w^Tx_i-y_i)^2 = \frac{1}{2n}||Xw-y||^2 = \frac{1}{2n}(Xw-y)^T(Xw-y) =
$$
$$
=\frac{1}{2n}((Xw)^TXw - (Xw)^Ty-y^TXw +y^Ty) = \frac{1}{2n}((Xw)^TXw - 2y^TXw +y^Ty) 
$$
$$
= \frac{1}{2n}(w^TX^TXw - 2y^TXw +y^Ty)
$$



$$
\frac{\partial E}{\partial w} = 0 \Rightarrow \frac{1}{2n}(2X^TXw-2X^Ty) = \frac{1}{n}(X^TXw-X^Ty)
$$

$$
X^TXw = X^Ty
$$
$$
w=(X^TX)^{-1}X^Ty
$$

Производная квадратичной формы: $\nabla_w(w^TAw)$=$2Aw$ если $A$=$A^T$ \
Производная линейной формы: $\nabla_w(b^Tw)$=$b$

-----

## Singular Value Decomposition

$$
w=(X^TX)^{-1}X^Ty
$$

$$
X = U \Sigma V^T
$$


$U$ - Ортогональная матрица левых сингулярных векторов ( $U^TU$=$I$ )\
$\Sigma$ - Диагональная матрица сингулярных значений\
$V^T$ - Ортогональная матрица правых сингулярных векторов ( $V^TV$=$I$ )


$$
w = (U^T \Sigma^TVU \Sigma V^T)^{-1}U^T \Sigma^TVy
$$
$$
w = (\Sigma^T \Sigma)^{-1} \Sigma^T V U^T y
$$
$$
w = V\Sigma^+U^Ty
$$

---

In [8]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, root_mean_squared_error

In [141]:
X, y = make_regression(n_samples=2_000, n_features=10, n_informative=8, noise=20, random_state=1)
X_train, X_test, y_train,y_test = train_test_split(X, y, train_size=0.8)

In [142]:
np.linalg.inv(X.T@X)@X.T@y

array([72.32560025, -0.31807231, 71.41889999, -0.34400868,  9.06165822,
       28.43109493, 29.72539612, 39.86434335, 49.4886773 ,  6.35910271])

In [143]:
np.linalg.lstsq(X, y, rcond=-1)[0]

array([72.32560025, -0.31807231, 71.41889999, -0.34400868,  9.06165822,
       28.43109493, 29.72539612, 39.86434335, 49.4886773 ,  6.35910271])

In [144]:
from sklearn.linear_model import LinearRegression

In [145]:
linreg = LinearRegression()
linreg.fit(X, y)
linreg.coef_

array([72.30067198, -0.3038456 , 71.43551651, -0.34565481,  9.09181283,
       28.44528909, 29.72228532, 39.86262761, 49.4887416 ,  6.36459013])

**Выражденная матрица**

In [159]:
x1 = np.arange(1, 101).reshape(-1, 1)
x2 = (x1*2).reshape(-1, 1)
x3 = (x2*3).reshape(-1, 1)
x4 = (x2*3 + x3*4).reshape(-1, 1)

X = np.hstack((x1, x2))
X = np.hstack((X, x3))
X = np.hstack((X, x4))

y = np.arange(1, 200, 2)

In [172]:
np.linalg.det(X.T@X)

0.0

In [160]:
np.linalg.inv(X.T@X)@A.T@y

LinAlgError: Singular matrix

In [161]:
np.linalg.pinv(X.T@X)@X.T@y

array([0.00210954, 0.00421907, 0.01265722, 0.06328612])

In [162]:
U, S, VT = np.linalg.svd(X, full_matrices=False)
S = np.diag(S)

In [169]:
VT.T @ np.linalg.pinv(S.T@S)@S.T @ U.T@y

array([0.00210954, 0.00421907, 0.01265722, 0.06328612])

In [170]:
linreg.fit(X, y)
linreg.coef_

array([0.0021254 , 0.0042508 , 0.01275239, 0.06376196])

In [174]:
class LeastSquares():
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None
        
    def fit(self, mode='svd'):
        pass
        
    def predict(self):
        pass

# Gradient Descent

Итеративный метод минимизации функции потерь

----
$$
\frac{\partial L}{\partial w} = \nabla\frac{1}{2n}|| Xw - y ||^2 = \frac{1}{n}X^T(Xw-y)
$$

$$
w = w_{i-1} - \eta \frac{\partial L}{\partial w}
$$

---


In [35]:
class BatchGD():
    def __init__(self, eps=0.1, learning_rate=0.01, max_iter=1000):
        self.eps = eps
        self.learning_rate = learning_rate
        self.coef_ = None
        self.intercept_ = None
        
    @staticmethod
    def grad(X, y, w):
        gradient_vector = 1/len(X) * X.T @ (X@w - y)
        return gradient_vector
        
    def fit(self, X, y, logging=False):
        # Вектор столбец
        y = y.reshape(-1, 1) 
        # Фиктивный признак единиц
        ones = np.ones(X.shape[0]).reshape(-1, 1) 
        X_ones = np.hstack((ones, X))
        
        # Случайная инициализация стартовых весов
        w = np.random.randn(X_ones.shape[1], 1)

        i=0
        
        for i in range(100):
            w = w - self.grad(X_ones, y, w)*self.eps
            
        self.coef_ = w.reshape(-1)[1:]
        self.intercept_ = w.reshape(-1)[0].reshape(1)

In [40]:
class StochasticGD():
    def __init__(self, learning_rate=0.1, max_iter=1000, n_epochs=50):
        self.learning_rate = learning_rate
        self.coef_ = None
        self.intercept_ = None
        self.n_epochs = n_epochs
        self.t1 = 5
        self.t2 = 50
        
    @staticmethod
    def learning_schedule(t, t1, t2):
        ls = t1 / (t + t2)
        return ls
        
    @staticmethod
    def grad(X, y, w):
        gradient_vector = 2/len(X) * X.T @ (X@w - y)
        return gradient_vector
        
    def fit(self, X, y, logging=False):
        # Вектор столбец
        y = y.reshape(-1, 1) 
        # Фиктивный признак единиц
        ones = np.ones(X.shape[0]).reshape(-1, 1) 
        X_ones = np.hstack((ones, X))
        m = len(X_ones)
        # Случайная инициализация стартовых весов
        w = np.random.randn(X_ones.shape[1], 1)
        
        for epoch in range(self.n_epochs):
            for i in range(m):
                random_index = np.random.randint(m)
                x_i = X_ones[random_index:random_index+1]
                y_i = y[random_index:random_index+1]
                eps = self.learning_schedule(epoch*m+i, self.t1, self.t2)
                w = w - self.grad(x_i, y_i, w)*eps   
            
        self.coef_ = w.reshape(-1)[1:]
        self.intercept_ = w.reshape(-1)[0].reshape(1)

In [41]:
class MiniBatchGD():
    def __init__(self, learning_rate=0.1, max_iter=1000, n_epochs=50, minibatch_size=20):
        self.learning_rate = learning_rate
        self.coef_ = None
        self.intercept_ = None
        self.n_epochs = n_epochs
        self.minibatch_size = minibatch_size
        self.t1 = 5
        self.t2 = 50
        
    @staticmethod
    def learning_schedule(t, t1, t2):
        ls = t1 / (t + t2)
        return ls
        
    @staticmethod
    def grad(X, y, w, minibatch_size):
        gradient_vector = 2/minibatch_size * X.T @ (X@w - y)
        return gradient_vector
        
    def fit(self, X, y, logging=False):
        # Вектор столбец
        y = y.reshape(-1, 1) 
        # Фиктивный признак единиц
        ones = np.ones(X.shape[0]).reshape(-1, 1) 
        X_ones = np.hstack((ones, X))
        m = len(X_ones)
        # Случайная инициализация стартовых весов
        w = np.random.randn(X_ones.shape[1], 1)
        t = 0
        for epoch in range(self.n_epochs):
            
            shuffled_indices = np.random.permutation(m)
            X_ones_shuffled = X_ones[shuffled_indices]
            y_shuffled = y[shuffled_indices]
            
            for i in range(0, m, self.minibatch_size):
                t+=1
                x_i = X_ones_shuffled[i:i+self.minibatch_size]
                y_i = y_shuffled[i:i+self.minibatch_size]
                
                eps = self.learning_schedule(t, self.t1, self.t2)
                w = w - self.grad(x_i, y_i, w, self.minibatch_size)*eps   
            
        self.coef_ = w.reshape(-1)[1:]
        self.intercept_ = w.reshape(-1)[0].reshape(1)

# L1 Regularization | Lasso

In [6]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV


# L2 Regularization | Ridge

In [7]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

# L1+L2 Regularization | ElastincNet

In [8]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV

# Regularization in Classification

In [9]:
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV